# QA中对于持仓成本的计算未考虑复权因素，这里增加了 `_get_hold_price` 方法来计算复权后的持仓成本

维护中的方法已移至 `finance_ai_py.quantaxis_ext`

In [75]:
import QUANTAXIS as QA
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

### 创建账户

股票普通账户

In [33]:
stock_account= QA.QA_Account(allow_t0=False,allow_margin=False,allow_sellopen=False,running_environment=QA.MARKET_TYPE.STOCK_CN)
# 重设账户初始资金为10万元
stock_account.reset_assets(100000)

### 初始化回测broker

In [34]:
Broker=QA.QA_BacktestBroker()

### 读取日线数据

000002在 2008-06 间进行过一次分红

公告日期	分红方案(每10股)	进度	除权除息日	股权登记日
2008-06-05	0	6	1	实施	2008-06-16	2008-06-13

In [35]:
data=QA.QA_fetch_stock_day_adv(['000002'],'2008-06-12','2008-06-18')

#### 这几天只做买入操作，之后计算均价用

In [37]:
i=0
for items in data.panel_gen:
    for item in items.security_gen:
        if i%2!=0:
            continue
        order = stock_account.send_order(
            code=item.code[0],
            time=item.date[0],
            amount=100,
            towards=QA.ORDER_DIRECTION.BUY,
            price=0,
            order_model=QA.ORDER_MODEL.CLOSE,
            amount_model=QA.AMOUNT_MODEL.BY_AMOUNT  # 按数量下单
            #在按金额下单的时候,应给予 money参数
            #在按数量下单的时候,应给予 amount参数
        )
        #print(item.to_json()[0])
        Broker.receive_order(QA.QA_Event(order=order, market_data=item))

        trade_mes = Broker.query_orders(stock_account.account_cookie, 'filled')
        res = trade_mes.loc[order.account_cookie, order.realorder_id]
        order.trade(res.trade_id, res.trade_price, res.trade_amount,
                    res.trade_time)
    stock_account.settle()
    i=i+1

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>


< QA_Order realorder_id Order_M9JTRjSI datetime:2008-06-12 15:00:00 code:000002 amount:100 price:17.35 towards:1 btype:stock_cn order_id:Order_M9JTRjSI account:Acc_YczXG2oq status:queued >

receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>


< QA_Order realorder_id Order_VBUcD7Hh datetime:2008-06-16 15:00:00 code:000002 amount:100 price:10.6 towards:1 btype:stock_cn order_id:Order_VBUcD7Hh account:Acc_YczXG2oq status:queued >

receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>


< QA_Order realorder_id Order_D5AXhNKe datetime:2008-06-18 15:00:00 code:000002 amount:100 price:10.27 towards:1 btype:stock_cn order_id:Order_D5AXhNKe account:Acc_YczXG2oq status:queued >

receive deal


### 账户信息

In [38]:
stock_account.message

{'account_cookie': 'Acc_YczXG2oq',
 'allow_sellopen': False,
 'allow_t0': False,
 'broker': 'backtest',
 'cash': [100000, 98260.0, 97195.0, 96163.0],
 'commission_coeff': 0.00025,
 'current_time': 'None',
 'end_date': '2008-06-18',
 'history': [['2008-06-12 00:00:00',
   '000002',
   17.35,
   100,
   98260.0,
   'Order_M9JTRjSI',
   'Order_M9JTRjSI',
   'Trade_ViH0JL5Y',
   'Acc_YczXG2oq',
   5,
   0,
   None,
   0],
  ['2008-06-16 00:00:00',
   '000002',
   10.6,
   100,
   97195.0,
   'Order_VBUcD7Hh',
   'Order_VBUcD7Hh',
   'Trade_XTj5rNYZ',
   'Acc_YczXG2oq',
   5,
   0,
   None,
   0],
  ['2008-06-18 00:00:00',
   '000002',
   10.27,
   100,
   96163.0,
   'Order_D5AXhNKe',
   'Order_D5AXhNKe',
   'Trade_ynpOBxYW',
   'Acc_YczXG2oq',
   5,
   0,
   None,
   0]],
 'init_assets': {'cash': 100000, 'hold': {}},
 'init_cash': 100000,
 'init_hold': {},
 'margin_level': {},
 'market_type': 'stock_cn',
 'portfolio_cookie': None,
 'quantaxis_version': '1.2.8.dev2',
 'running_environment'

In [39]:
stock_account.history_table

,datetime,code,price,amount,cash,order_id,realorder_id,trade_id,account_cookie,commission,tax,message,frozen
0,2008-06-12 00:00:00,000002,17.35,100,98260.0,Order_M9JTRjSI,Order_M9JTRjSI,Trade_ViH0JL5Y,Acc_YczXG2oq,5,0,None,0
1,2008-06-16 00:00:00,000002,10.60,100,97195.0,Order_VBUcD7Hh,Order_VBUcD7Hh,Trade_XTj5rNYZ,Acc_YczXG2oq,5,0,None,0
2,2008-06-18 00:00:00,000002,10.27,100,96163.0,Order_D5AXhNKe,Order_D5AXhNKe,Trade_ynpOBxYW,Acc_YczXG2oq,5,0,None,0


#### 持仓成本价

这里只是简单的求了平均，未考虑分红配送的因素

**应该重新取购买日期~当前日期间的数据后取 前复权 的价格为准**

In [40]:
stock_account.hold_price()
stock_account.history_table.price.mean()

code
000002    (12.74, 300.0)
dtype: object

12.74

In [41]:
stock_account.hold_table()

code
000002    300
Name: amount, dtype: int64

#### 可用持仓

In [42]:
stock_account.hold_available

code
000002    300
Name: amount, dtype: int64

In [43]:
stock_account.hold_time()

code
000002   1 days 09:30:00
dtype: timedelta64[ns]

#### hold_price函数实现

In [44]:
stock_account.history_table.set_index('datetime', drop=False).sort_index()

,datetime,code,price,amount,cash,order_id,realorder_id,trade_id,account_cookie,commission,tax,message,frozen
datetime,,,,,,,,,,,,,
2008-06-12 00:00:00,2008-06-12 00:00:00,000002,17.35,100,98260.0,Order_M9JTRjSI,Order_M9JTRjSI,Trade_ViH0JL5Y,Acc_YczXG2oq,5,0,None,0
2008-06-16 00:00:00,2008-06-16 00:00:00,000002,10.60,100,97195.0,Order_VBUcD7Hh,Order_VBUcD7Hh,Trade_XTj5rNYZ,Acc_YczXG2oq,5,0,None,0
2008-06-18 00:00:00,2008-06-18 00:00:00,000002,10.27,100,96163.0,Order_D5AXhNKe,Order_D5AXhNKe,Trade_ynpOBxYW,Acc_YczXG2oq,5,0,None,0


In [45]:
QA.QA_fetch_stock_day_adv(['000002'],'2008-06-12','2008-06-18').data.close

date        code  
2008-06-12  000002    17.35
2008-06-13  000002    16.50
2008-06-16  000002    10.60
2008-06-17  000002     9.95
2008-06-18  000002    10.27
Name: close, dtype: float64

In [46]:
QA.QA_fetch_stock_day_adv(['000002'],'2008-06-12','2008-06-18').to_hfq().data.close

date        code  
2008-06-12  000002    17.350000
2008-06-13  000002    16.500000
2008-06-16  000002    17.063415
2008-06-17  000002    16.017073
2008-06-18  000002    16.532195
Name: close, dtype: float64

In [47]:
QA.QA_fetch_stock_day_adv(['000002'],'2008-06-12','2008-06-18').to_qfq().data.close

date        code  
2008-06-12  000002    10.77803
2008-06-13  000002    10.25000
2008-06-16  000002    10.60000
2008-06-17  000002     9.95000
2008-06-18  000002    10.27000
Name: close, dtype: float64

In [87]:
def _get_qfq_history_table(account, enddate):
    """在history_table中增加前复权价格"""
    hold_df = account.history_table.set_index(
        'datetime', drop=False).sort_index()
    hold_df[['code', 'price']]
    hold_df['price_qfq'] = -1.0
    if not isinstance(enddate, str):
        enddate = QA.QAUtil.QADate.QA_util_datetime_to_strdate(enddate)
    for idx in hold_df.index:
        new_value = QA.QA_fetch_stock_day_adv(hold_df.loc[idx, 'code'], idx,
                                              enddate).to_qfq()
        hold_df.loc[idx, 'price_qfq'] = new_value.close[0]
    return hold_df


# _get_qfq_history_table(stock_account, '2008-06-18')


def _get_hold_price(account, datetime):
    """计算持仓成本  如果给的是日期,则返回当日开盘前的持仓
        **替代QAAccount.hold_price**,以前复权价格来计算
        Keyword Arguments:
            datetime {[type]} -- [description] (default: {None})

        Returns:
            [type] -- [description]
        """

    def weights(x):
        if sum(x['amount']) != 0:
            return np.average(
                x['price_qfq'], weights=x['amount'], returned=True)
        else:
            return np.nan

    df = _get_qfq_history_table(account, datetime)

    if datetime is None:
        return df.groupby('code').apply(weights).dropna()
    else:
        return df.loc[:datetime].groupby('code').apply(weights).dropna()

### 当前持仓成本

In [88]:
_get_hold_price(stock_account,'2008-06-18')
'以前复权价计算的持仓成本'
stock_account.hold_price('2008-06-18')
'标准持仓成本（只计算了当时购买价格，未考虑复权）'
stock_account.hold_price()

code
000002    (10.689015151515152, 200.0)
dtype: object

code
000002    (13.975, 200.0)
dtype: object

code
000002    (12.74, 300.0)
dtype: object

12.74

In [90]:
'以前复权价计算的持仓成本'
_get_hold_price(stock_account, '2008-06-19')
'标准持仓成本（只计算了当时购买价格，未考虑复权）'
stock_account.hold_price('2008-06-19')

'以前复权价计算的持仓成本'

code
000002    (10.549343434343434, 300.0)
dtype: object

'标准持仓成本（只计算了当时购买价格，未考虑复权）'

code
000002    (12.74, 300.0)
dtype: object